# ELT GPS Retroescavadeira
## 1. Extract
1.1. If dont have parquet file it will come from gsheet
## 2.Load
2.1. Load and convert in df pandas
## 3. Transform
3.1. Normalize de dataframe


Nestre exemplo os dados coletados em 2022 fornecidos pelo gps da retroscavadeira JCB 3CX, foram armazenados em um unico arquivo em formato de gsheet. Este estudo tem por finalidade **obter informação** utilizando tecnica da engenharia de dados

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import extrair as extrair
import fx_data as fx_data
import filtros as filtros

In [3]:
#silver
#tranformar
df = pd.read_parquet('data/bronze_jcb_relatorio_2022.parquet',engine='pyarrow')
remover_colunas = ['id','hyperlink','maps_google_url']
df1 = fx_data.df_bronze_to_silver_gps(df=df,remover_colunas=remover_colunas)
df1.head(5)

,data_hora,atividade,gps_ativo,data,hora,nome_dia,lat,lon,lat_lon,lat_ant,lon_ant,raio_m,motor_ligado
0,2022-01-01 11:56:00,estado_activo,False,2022-01-01,11,saturday,-25.570758,-49.196145,-25.5707582|-49.1961449,NaN,NaN,NaN,False
1,2022-01-01 11:56:00,ligado,False,2022-01-01,11,saturday,-25.570758,-49.196145,-25.5707582|-49.1961449,-25.570758,-49.196145,0.0,False
19,2022-01-01 12:56:00,prestes_a_entrar_em_estado_de_descanso,True,2022-01-01,12,saturday,-25.570774,-49.196087,-25.5707737|-49.1960868,-25.570770,-49.196085,0.0,False
21,2022-01-02 00:56:00,estado_activo,False,2022-01-02,0,sunday,-25.570774,-49.196087,-25.5707737|-49.1960868,-25.570774,-49.196087,0.0,False
22,2022-01-02 00:56:00,ligado,False,2022-01-02,0,sunday,-25.570774,-49.196087,-25.5707737|-49.1960868,-25.570774,-49.196087,0.0,False


In [4]:
df2 = df1.query('motor_ligado==True')
display(df2.head(5))
display(df2['nome_dia'].value_counts())
display(df2['hora'].value_counts().sort_index())
df3 = df1.query('(motor_ligado==True) and (nome_dia=="sunday")')
display(df3['hora'].value_counts().sort_index())
display(df3.head(25))

,data_hora,atividade,gps_ativo,data,hora,nome_dia,lat,lon,lat_lon,lat_ant,lon_ant,raio_m,motor_ligado
70,2022-01-05 08:37:00,chave_ligada,False,2022-01-05,8,wednesday,-25.570799,-49.196091,-25.5707991|-49.1960913,-25.570799,-49.196091,0.0,True
71,2022-01-05 08:37:00,ligado,False,2022-01-05,8,wednesday,-25.570799,-49.196091,-25.5707991|-49.1960913,-25.570799,-49.196091,0.0,True
72,2022-01-05 08:37:00,arranque_do_motor,False,2022-01-05,8,wednesday,-25.570799,-49.196091,-25.5707991|-49.1960913,-25.570799,-49.196091,0.0,True
89,2022-01-05 13:51:00,chave_ligada,False,2022-01-05,13,wednesday,-25.570302,-49.196398,-25.5703022|-49.1963979,-25.570302,-49.196398,0.0,True
90,2022-01-05 13:51:00,ligado,False,2022-01-05,13,wednesday,-25.570302,-49.196398,-25.5703022|-49.1963979,-25.570302,-49.196398,0.0,True


friday       7701
monday       7600
thursday     7584
wednesday    7307
tuesday      7240
saturday     3084
sunday        245
Name: nome_dia, dtype: Int64

5        9
6      146
7     4759
8     5285
9     4130
10    5212
11    2972
12    1637
13    4717
14    4089
15    3711
16    2555
17    1129
18     328
19      57
20      17
21       8
Name: hora, dtype: int64

6      2
7     11
8     44
9     19
10    57
11    24
13    30
14    18
15     2
16     9
17    17
18     7
19     5
Name: hora, dtype: int64

,data_hora,atividade,gps_ativo,data,hora,nome_dia,lat,lon,lat_lon,lat_ant,lon_ant,raio_m,motor_ligado
27161,2022-07-17 08:01:00,chave_ligada,False,2022-07-17,8,sunday,-26.602457,-51.096624,-26.6024571|-51.0966244,-26.602457,-51.096624,0.0,True
27162,2022-07-17 08:01:00,arranque_do_motor,False,2022-07-17,8,sunday,-26.602457,-51.096624,-26.6024571|-51.0966244,-26.602457,-51.096624,0.0,True
34953,2022-09-04 11:24:00,chave_ligada,False,2022-09-04,11,sunday,-26.603237,-51.097721,-26.6032373|-51.0977211,-26.603237,-51.097721,0.0,True
34954,2022-09-04 11:24:00,ligado,False,2022-09-04,11,sunday,-26.603237,-51.097721,-26.6032373|-51.0977211,-26.603237,-51.097721,0.0,True
34955,2022-09-04 11:24:00,arranque_do_motor,False,2022-09-04,11,sunday,-26.603237,-51.097721,-26.6032373|-51.0977211,-26.603237,-51.097721,0.0,True
34957,2022-09-04 11:26:00,em_andamento,True,2022-09-04,11,sunday,-26.605413,-51.098191,-26.6054133|-51.0981914,-26.605413,-51.098191,0.0,True
34958,2022-09-04 11:26:00,em_andamento,True,2022-09-04,11,sunday,-26.605532,-51.098401,-26.6055325|-51.0984013,-26.605413,-51.098191,25.0,True
34959,2022-09-04 11:29:00,parado,True,2022-09-04,11,sunday,-26.604774,-51.099611,-26.6047735|-51.0996108,-26.605532,-51.098401,147.0,True
34960,2022-09-04 11:36:00,em_andamento,True,2022-09-04,11,sunday,-26.604891,-51.099347,-26.6048906|-51.0993469,-26.604774,-51.099611,29.0,True
34964,2022-09-04 11:40:00,chave_ligada,True,2022-09-04,11,sunday,-26.605310,-51.099269,-26.60531|-51.0992694,-26.605297,-51.099276,2.0,True


In [5]:
df3 = df1[df1['atividade'] == 'chave_ligada'].groupby(['data', 'atividade'])['hora'].first().reset_index()
display(df3)
display(df3['hora'].value_counts().sort_index())
df3['data'].nunique()

,data,atividade,hora
0,2022-01-05,chave_ligada,8
1,2022-01-07,chave_ligada,11
2,2022-01-10,chave_ligada,8
3,2022-01-12,chave_ligada,8
4,2022-01-20,chave_ligada,16
...,...,...,...
464,2023-08-25,chave_ligada,7
465,2023-08-28,chave_ligada,7
466,2023-08-29,chave_ligada,7
467,2023-08-30,chave_ligada,8


5       4
6      40
7     339
8      36
9       9
10     13
11      7
12      3
13      6
14      1
15      1
16      2
17      5
18      1
19      2
Name: hora, dtype: int64

469

In [8]:
#salvar 
caminho = 'C:\\Users\\User\\Documents\\GitHub\\gps_maquina\\data\\' 
arquivo = 'silver_jcb_relatorio_2022.parquet'
df1.to_parquet(caminho+arquivo, index=False)

In [7]:
#%%timeit
hora = 15
atividade = 'arranque_do_motor'
atividades = ['arranque_do_motor','paragem_do_motor']
consultas = [
    '(atividade == @atividade) and (hora==12)',
    '(atividade == @atividade) & (hora==12) and (nome_dia=="sunday")',
    '(atividade == @atividade) & (hora==12) or (nome_dia=="sunday") or (nome_dia=="saturday")',
    '(atividade == @atividades) & (hora==12)',
]

'premissas de comportamento fora da normalidade'
comportamentos = [
    {'nome':'chave ligada a noite','query':'(atividade == "chave_ligada") and (hora>18)'},
    {'nome':'chave ligada madrugada','query':'(atividade == "chave_ligada") and (hora>18)'}
]

df1.query(consultas[3])

SyntaxError: invalid syntax (1205836937.py, line 14)

### encontrar o quando a maquina esteve em operação
1. horario
2. dia da semana

In [ ]:
def turno_atividade(atividade:str,df:pd.DataFrame)->dict:
    """
    Encontrar o horario de operação de uma atividade:   
    """
    df1 = df.query(f'atividade =="{atividade}"')
    total_linhas = df1.shape[0]
    
    turnos = {
        'magrudada':{'query':'0<=hora<=6','qtd':0,'%':0},
        'dia':{'query':'7<=hora<=17','qtd':0,'%':0},
        'noite':{'query':'18<=hora<=23','qtd':0,'%':0},
        }
    for k,v in turnos.items():
        turnos[k]['qtd'] = len(df1.query(v['query']))
        turnos[k]['%'] = round((turnos[k]['qtd']/total_linhas)*100,2)
        turnos[k]['ativo']= 1 if turnos[k]['%']>5 else 0
    
    df1['hora'].value_counts().sort_index(ascending=True).plot(kind='bar')
    
    turno_ativo = {k: v for k, v in turnos.items() if v['ativo'] > 0}
    
    return turno_ativo
turnos_arranque_motor = turno_atividade(atividade='arranque_do_motor',df=df1)
display(turnos_arranque_motor)

In [ ]:
def dia_semana_atividade(atividade:str,df:pd.DataFrame)->dict:
    df1 = df.query(f'atividade =="{atividade}"')
    total_linhas = df1.shape[0]
     
    dic = {}
    dias_semana = df1['nome_dia'].unique().tolist()
    for dia in dias_semana:
        
        dic[dia]=len(df1.query(f'nome_dia =="{dia}"'))
        dic[dia] =round(dic[dia]/total_linhas,2)
        dic[dia] = int(dic[dia]*100)
        dia_ativo = {day: value for day, value in dic.items() if value > 5}
        
        df1['nome_dia'].value_counts().sort_index(ascending=True).plot(kind='barh')
        
    return dia_ativo

dia_arranque_motor = dia_semana_atividade(atividade='arranque_do_motor',df=df1)
    
display(dia_arranque_motor)
    

### quais os locais a maquina esteve desativada

In [ ]:
df2 = df1.loc[(df1['hora'] > 20) | (df1['hora'] < 4)| (df1['atividade'] =='arranque_do_motor')]
display(df2.head(5))
unicos_lat_lon = df2['lat_lon'].unique()
display(unicos_lat_lon)
display(len(unicos_lat_lon))


### Grafico de Posicionamento x,y
Utilizando as coordenadas lat e lon e convertendo em um grafico de x,y para entender as regiões nas quais o equimamento esteve mais presente durante o periodo

In [ ]:
df1.plot.scatter(x='lon',y='lat')#, marker='o', color='blue', s=50, alpha=0.5)

# Add labels and title
#plt.xlabel('Latitude')
#plt.ylabel('Longitude')
#plt.title('Scatter Plot of Latitude vs. Longitude')

# Show the plot
#plt.show()

In [ ]:
df2.plot.scatter(x='lat', y='lon')#, marker='o', color='blue', s=50, alpha=0.5)

### calcular a distancia a partir de pontos conhecidos


In [ ]:
dic = extrair.json_to_dic('variaveis.json')
#display(dic['locais'])
df_locais = pd.DataFrame(dic['locais'])

display(df_locais)

## grafico por mes!!

In [ ]:
inicio=5
isinstance(inicio, datetime)